The objective of this project assignment is to develop a machine learning model that predicts future air quality levels based on historical air quality data and relevant environmental factors. Students will gain practical experience in time series analysis, feature engineering, model selection, and evaluation for air quality forecasting.

The dataset consists of hourly atmospheric measurements from 12 cities in Beijing, covering the period from March 1st, 2013, to February 28th, 2017. The target variables are PM2.5, PM10, SO2, NO2, CO, and O3, along with independent variables like temperature, pressure, dew point temperature, rainfall, wind speed, and wind direction.

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
import statsmodels.api as sm
from pmdarima import auto_arima

## LOADING DATA AND EDA

In [5]:
# Load Data

df = pd.read_csv('dataset.csv')

print("Shape:", df.shape)
print("\nColumns:\n", df.columns.tolist())

display(df.head())
display(df.tail())

# Basic structural info
df.info()

# Quick sanity checks
print("\nNumber of duplicate rows:", df.duplicated().sum())
print("Number of stations:", df["station"].nunique())
print("Stations:", sorted(df["station"].unique())[:12], "...")


Shape: (420768, 18)

Columns:
 ['No', 'year', 'month', 'day', 'hour', 'PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'wd', 'WSPM', 'station']


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,4.0,4.0,4.0,7.0,300.0,77.0,-0.7,1023.0,-18.8,0.0,NNW,4.4,Aotizhongxin
1,2,2013,3,1,1,8.0,8.0,4.0,7.0,300.0,77.0,-1.1,1023.2,-18.2,0.0,N,4.7,Aotizhongxin
2,3,2013,3,1,2,7.0,7.0,5.0,10.0,300.0,73.0,-1.1,1023.5,-18.2,0.0,NNW,5.6,Aotizhongxin
3,4,2013,3,1,3,6.0,6.0,11.0,11.0,300.0,72.0,-1.4,1024.5,-19.4,0.0,NW,3.1,Aotizhongxin
4,5,2013,3,1,4,3.0,3.0,12.0,12.0,300.0,72.0,-2.0,1025.2,-19.5,0.0,N,2.0,Aotizhongxin


,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
420763,35060,2017,2,28,19,11.0,32.0,3.0,24.0,400.0,72.0,12.5,1013.5,-16.2,0.0,NW,2.4,Wanshouxigong
420764,35061,2017,2,28,20,13.0,32.0,3.0,41.0,500.0,50.0,11.6,1013.6,-15.1,0.0,WNW,0.9,Wanshouxigong
420765,35062,2017,2,28,21,14.0,28.0,4.0,38.0,500.0,54.0,10.8,1014.2,-13.3,0.0,NW,1.1,Wanshouxigong
420766,35063,2017,2,28,22,12.0,23.0,4.0,30.0,400.0,59.0,10.5,1014.4,-12.9,0.0,NNW,1.2,Wanshouxigong
420767,35064,2017,2,28,23,13.0,19.0,4.0,38.0,600.0,49.0,8.6,1014.1,-15.9,0.0,NNE,1.3,Wanshouxigong


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 420768 entries, 0 to 420767
Data columns (total 18 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   No       420768 non-null  int64  
 1   year     420768 non-null  int64  
 2   month    420768 non-null  int64  
 3   day      420768 non-null  int64  
 4   hour     420768 non-null  int64  
 5   PM2.5    412029 non-null  float64
 6   PM10     414319 non-null  float64
 7   SO2      411747 non-null  float64
 8   NO2      408652 non-null  float64
 9   CO       400067 non-null  float64
 10  O3       407491 non-null  float64
 11  TEMP     420370 non-null  float64
 12  PRES     420375 non-null  float64
 13  DEWP     420365 non-null  float64
 14  RAIN     420378 non-null  float64
 15  wd       418946 non-null  object 
 16  WSPM     420450 non-null  float64
 17  station  420768 non-null  object 
dtypes: float64(11), int64(5), object(2)
memory usage: 57.8+ MB

Number of duplicate rows: 0
Number of statio

## Load + first look (what matters)
- The dataset loaded successfully and contains multiple years of **hourly** observations.
- Time is currently split across columns (`year`, `month`, `day`, `hour`) rather than a single datetime index — we’ll fix that soon.
- `station` is a categorical column identifying the monitoring location.
- Next we’ll quantify missingness and get a first sense of distributions and weird values.


In [6]:
# Missingness summary
missing_count = df.isna().sum().sort_values(ascending=False)
missing_pct = (df.isna().mean() * 100).sort_values(ascending=False)

missing_summary = pd.DataFrame({
    "missing_count": missing_count,
    "missing_pct": missing_pct
})

display(missing_summary[missing_summary["missing_count"] > 0])


,missing_count,missing_pct
CO,20701,4.919813
O3,13277,3.155421
NO2,12116,2.879497
SO2,9021,2.143937
PM2.5,8739,2.076916
PM10,6449,1.532674
wd,1822,0.433018
DEWP,403,0.095777
TEMP,398,0.094589
PRES,393,0.093401


## Missingness check — key takeaways

- Missing values exist mostly in the **pollutant target columns**, especially:
  - **CO (~4.92%)** — highest missingness
  - **O3 (~3.16%)**
  - **NO2 (~2.88%)**
  - **SO2 (~2.14%)**
  - **PM2.5 (~2.08%)**
  - **PM10 (~1.53%)**

- Meteorological variables have **very low missingness** (all under ~0.10%), which is great — they won’t be the main cleaning headache.

- Wind direction **wd** has a small amount missing (~0.43%). Since it’s categorical, we should treat it differently from numeric columns.

### Why this matters for forecasting
Time-series models (including ARIMA) generally dislike gaps. For hourly pollutant series, the most defensible approach is usually:
- **time-aware interpolation** for numeric variables (especially meteorology),
- and for pollutants: either **interpolate** (if gaps are small/short) or **drop only the missing rows in the final modeling subset** after filtering to one station/target.

We’ll decide the exact strategy after we:
1) convert to a proper datetime index,
2) focus on a single station + target series,
3) check whether missingness is scattered or occurs in blocks.


In [8]:
# Numeric distribution summary with robust percentiles (includes 25% and 75%)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

summary = df[num_cols].describe(percentiles=[0.01, 0.05, 0.25, 0.5, 0.75, 0.95, 0.99]).T
summary["IQR"] = summary["75%"] - summary["25%"]

display(summary.sort_values("99%", ascending=False))


,count,mean,std,min,1%,5%,25%,50%,75%,95%,99%,max,IQR
No,420768.0,17532.500000,10122.116943,1.0000,351.0,1754.0,8766.75,17532.5,26298.25,33311.0,34714.0,35064.0,17531.5
CO,400067.0,1230.766454,1160.182716,100.0000,100.0,200.0,500.00,900.0,1500.00,3500.0,6000.0,10000.0,1000.0
year,420768.0,2014.662560,1.177198,2013.0000,2013.0,2013.0,2014.00,2015.0,2016.00,2016.0,2017.0,2017.0,2.0
PRES,420375.0,1010.746982,10.474055,982.4000,990.4,994.7,1002.30,1010.4,1019.00,1027.9,1032.5,1042.8,16.7
PM10,414319.0,104.602618,91.772426,2.0000,5.0,10.0,36.00,82.0,145.00,279.0,422.0,999.0,109.0
PM2.5,412029.0,79.793428,80.822391,2.0000,3.0,6.0,20.00,55.0,111.00,242.0,370.0,999.0,91.0
O3,407491.0,57.372271,56.661607,0.2142,1.0,2.0,11.00,45.0,82.00,177.0,245.0,1071.0,71.0
NO2,408652.0,50.638586,35.127912,1.0265,2.0,8.0,23.00,43.0,71.00,117.0,157.0,290.0,48.0
SO2,411747.0,15.830835,21.650603,0.2856,2.0,2.0,3.00,7.0,20.00,60.0,104.0,500.0,17.0
TEMP,420370.0,13.538976,11.436139,-19.9000,-7.9,-4.2,3.10,14.5,23.30,30.6,33.8,41.6,20.2
